## <center><strong>Notebook describtion & Our <span style= 'color: #51fcc6'>Questions</span>

In this notebook we are going to create the charts that we are going to use in the final pdf report and the tools we are going<br>
to use are matplotlib and seaborn.

#### **Questions we are going to awnser:**
1. How many jobs are available per country and per profession on Likedin.

2. How many profiles have earned money on UpWork Vs Guru per 1,000 profile.

3. Average salary per profession and per country on LinkedIn.

4. What's Top 5 people in earnings amount most frequent words in thier describtions.<sub> *(In upwork</sub>

5. What's Worst 5 people in earnings amount most frequent words in thier describtions.<sub> *(In upwork)</sub>

6. What are the most appending skills in the job titels per profession.<sub> *(In upwork & LinkedIn)</sub>

7. What are the Top 10 skills in earnings. <sub> *(In upwork & LinkedIn)</sub>

8. Correlation between Earnings amount, Years of Exp, Hour rate & Feedback in Upwork.

9. Does the Consultations offers affect the Earnings amount strongly.

10. Top 3 countries in total freelancers earnings Vs Top 3 countries in freelancers count <sub> *(In upwork & guru)</sub>

11. Jobs count per Jobs location per profession.

12. Where are the most job loactions in general per profession.

13. What's the average salary per proffision & per country.

14. What's the most common Employment type per data jobs in general.

15. Jobs count vs listing dates per proffision to see thier growth trends.

16. most common Earning abount, hour rate, exp years per proffision in Upwork.

#### **Important note:**
I am going to use  **`pandasql`** to read the data from the dfs so I can create plots more easly using SQL<br>
queries inside python.

## <center><strong>Importing the <span style= 'color: #48e0dc'>Packeges 

In [1]:
import os
import warnings
import matplotlib
import pandasql           as ps
import pandas             as pd
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt

from google.cloud         import bigquery
from wordcloud            import WordCloud
from collections          import Counter

In [2]:
%matplotlib inline
plt.rcParams['font.family'] = 'Candara'

warnings.filterwarnings('ignore')

credentials_path :str = '../credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

FONT             :int  = 17
COLORS           :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                          'uint16', 'uint32', 'uint64']

## <center><strong>Loading the data & basic data <span style= 'color: #5cd3f7'>Cleaning

In [23]:
project_id :str = 'data-jobs-analysis-db'
dataset_id :str = 'data_jobs_analysis_db'

query_pandas  = lambda q: ps.sqldf(q, globals())
client        = bigquery.Client(project= project_id)

linkedin_jobs    = client.query(f'SELECT * FROM {dataset_id}.linkedin_jobs_new;').to_dataframe()
upwork_profiles  = client.query(f'SELECT * FROM {dataset_id}.upwork_profiles_new;').to_dataframe()
guru_profiles    = client.query(f'SELECT * FROM {dataset_id}.guru_profiles_new;').to_dataframe()

In [69]:
cm = sns.light_palette(COLORS[0], as_cmap=True)

linkedin_jobs.drop(['describtion', 'jobs_links', 'jobs_titles', 'skills']
                   , axis= 1).sample(5).style.background_gradient(cmap= cm)

,jobs_locations,listing_dates,companies_names,location_type,employment_type,industry,reqierd_credential,country,job_title,total_jobs,salary,programming_lang,exp_years,per_hour_or_year
1262,"San Francisco, CA",2023-05-25,Cypress HCM,Remote,CONTRACTOR,"Technology, Information and Internet",bachelor degree,United States,Data analyst,202000,nan,['sql' 'scala' 'python'],5.166667,nan
2031,"Oakland, CA",2023-06-18,Kaiser Permanente,Remote,FULL_TIME,Hospitals and Health Care,bachelor degree,United States,Data analyst,202000,149835.000000,['sql' 'scala' 'python'],nan,per year
2910,"South San Francisco, CA",2023-05-26,Recruiting from Scratch,Remote,FULL_TIME,Staffing and Recruiting,bachelor degree,United States,ML developer,2000,182500.000000,['scala' 'c++' 'python'],nan,per year
2128,"Cupertino, CA",2023-06-15,Hermitage Infotech,Remote,FULL_TIME,IT Services and IT Consulting,bachelor degree,United States,Data analyst,202000,nan,['sql' 'python'],7.000000,nan
1379,"San Francisco, CA",2023-06-07,CoStar Group,Remote,FULL_TIME,"IT Services and IT Consulting,Leasing Non-residential Real Estate,Research Services",bachelor degree,United States,Data scientist,167000,nan,['scala' 'python'],1.000000,nan


In [70]:
cm = sns.light_palette(COLORS[2], as_cmap=True)

upwork_profiles.drop(['describtion', 'skills']
                   , axis= 1).sample(5).style.background_gradient(cmap= cm)

,feedback,name,job_title,country,consultations_offers,data_job_title,earnings_amount_new,hour_rate_new,exp_years
911,nan,Hammad S.,OpenAi ChatGPT/Midjourney /DALL-E 2 Expert.,Pakistan,Doesn't offer consultations,Machine Learning,1000,25.000000,nan
464,100.000000,Sudhir J.,Certified Excel VBA / Data entry Specialist,India,Doesn't offer consultations,Data entry,8000,6.000000,8.000000
336,92.000000,Shams U.,List Building/Data Entry/Lead Generation/Data Mining/Prospect List,Pakistan,Doesn't offer consultations,Data entry,20000,12.000000,nan
384,93.000000,Md. Habibur Rahman S.,PhD Expert Writer|Thesis|Data Analyst|Research Paper|Machine Learning,Bangladesh,Offers consultations,Data Engineering,30000,30.000000,nan
124,100.000000,Nurettin Tevfik E.,Data Scientist,Turkey,Doesn't offer consultations,Data scientist,,50.000000,nan


In [71]:
cm = sns.light_palette(COLORS[4], as_cmap=True)

guru_profiles.drop(['describtion', 'skills']
                   , axis= 1).sample(5).style.background_gradient(cmap= cm)

,name,job_title,addresse,minimum_pay,data_job_title,earnings_amount_new,hour_rate_new,feedback_new
1558,Bose Amit,Data Analyst,"Bengaluru, Karnataka, India",100.000000,Data Analyst,0,15100.000000,0.000000
1421,Mohit DS,"Expert Data Analyst - Python , R Studio","Bangalore, Karnataka, India",50.000000,Data Analyst,0,3050.000000,0.000000
921,Ramzan12,Python Machine Learning NLP,"Bahawalpur, Punjab, Pakistan",25.000000,Machine Learning,0,525.000000,0.000000
1419,Aaron Skopeal,Data analyst and visualization expert,"Helsinki, Southern Finland, Finland",50.000000,Data Analyst,0,4550.000000,0.000000
990,Snehal Samaria,Machine Learning,"Ajmer, Rajasthan, India",25.000000,Machine Learning,0,825.000000,0.000000


#### **Basic cleaning:**

In [67]:
upwork_profiles.drop(['Unnamed__0', 'new_columns'], axis= 1, inplace= True)
guru_profiles.drop(['Unnamed__0', 'new_columns'], axis= 1, inplace= True)

linkedin_jobs.drop(['jobs_links'], axis= 1, inplace= True)

linkedin_jobs['salary'] = linkedin_jobs['salary'].where((linkedin_jobs.salary < 70) |
                                                        (linkedin_jobs.salary > 10_000) &
                                                        (linkedin_jobs.salary > 5))

per_hour_or_year_map = lambda salary: 'per year' if salary > 10_000 else 'per hour' if salary < 100 else np.nan
linkedin_jobs['per_hour_or_year'] = linkedin_jobs['salary'].apply(per_hour_or_year_map)

## <center><strong><span style= 'color: #5cd3f7'>Visualizing</span> the data (part 1)

In [74]:
query_pandas('''SELECT total_jobs,
                       country,
                       job_title

                FROM linkedin_jobs
                GROUP BY job_title, country''')

,total_jobs,country,job_title
0,156000,European Union,Data analyst
1,202000,United States,Data analyst
2,39000,European Union,Data engineer
3,76000,United States,Data engineer
4,264000,European Union,Data entry
5,786000,United States,Data entry
6,142000,European Union,Data scientist
7,167000,United States,Data scientist
8,989,European Union,ML developer
9,2000,United States,ML developer
